In [6]:
import pandas as pd

In [17]:
from itertools import product

#valid_q1 = ['birthyear', 'birthplace']
#valid_q2 = ['capital', 'uspresident', 'nobelLiterature', 'masterchamp']

#valid_q1 = ['birthyear']
#valid_q2 = ['nobelLiterature']

valid_q1 = ['birthplace']
valid_q2 = ['callingcode']

valid_all = []

for q1, q2 in product(valid_q1, valid_q2):
    valid_all.append('%s_%s' % (q1, q2))

print(valid_all)
    

['birthplace_callingcode']


In [18]:
import json
import math

with open('compositional_celebrities.json') as f_cc:
    cc_dset = json.load(f_cc)

df = pd.DataFrame(cc_dset['data'])

def remove_lists(ex, cols = ['A1', 'A2', 'Answer', 'capital']):
    for c in cols:
        if c not in ex:
            print(ex)
            continue
            
        if isinstance(ex[c], float) and math.isnan(ex[c]):
            continue

        ex[c] = ex[c][0]
    return ex

def filter_simple(ex):
    category_name = ex['category']

    return category_name in valid_all

df = df.apply(remove_lists, axis=1)

df = df[df.apply(filter_simple, axis=1)]

df = df.sample(frac=1)

df.head()

,Q1,A1,Q2,A2,Question,Answer,birth_country,capital,person,category,...,currency,currency_short,currency_symbol,jpn_common_name,spa_common_name,rus_common_name,est_common_name,urd_common_name,calling_code,birth_date
6375,What is the birthplace (country only) of Anast...,Nicaragua,What is the calling code of Nicaragua?,+505,What is the calling code of the birthplace of ...,+505,Nicaragua,NaN,Anastasio Somoza Debayle,birthplace_callingcode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[+505],NaN
6137,What is the birthplace (country only) of Jigme...,Bhutan,What is the calling code of Bhutan?,+975,What is the calling code of the birthplace of ...,+975,Bhutan,NaN,Jigme Khesar Namgyel Wangchuck,birthplace_callingcode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[+975],NaN
6126,What is the birthplace (country only) of Fazlu...,Bangladesh,What is the calling code of Bangladesh?,+880,What is the calling code of the birthplace of ...,+880,Bangladesh,NaN,Fazlur Rahman Khan,birthplace_callingcode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[+880],NaN
6258,What is the birthplace (country only) of Ricar...,Guatemala,What is the calling code of Guatemala?,+502,What is the calling code of the birthplace of ...,+502,Guatemala,NaN,Ricardo Arjona,birthplace_callingcode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[+502],NaN
6327,What is the birthplace (country only) of Abdul...,Kuwait,What is the calling code of Kuwait?,+965,What is the calling code of the birthplace of ...,+965,Kuwait,NaN,Abdullah Al Mutairi,birthplace_callingcode,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[+965],NaN


In [20]:
column_name_map = {
    'birthplace': 'birth_country',
    'callingcode': 'calling_code'
}

label_space_all = {}

for q in valid_q1 + valid_q2:
    col = q
    if q in column_name_map:
        col = column_name_map[col]
    
    print(df.columns)

    label_space = df[col].dropna()

    label_space_all[q] = label_space.tolist()

print(label_space_all.keys())

Index(['Q1', 'A1', 'Q2', 'A2', 'Question', 'Answer', 'birth_country',
       'capital', 'person', 'category', 'person_id', 'rounded_lat',
       'rounded_lng', 'tld', 'ccn3', 'currency', 'currency_short',
       'currency_symbol', 'jpn_common_name', 'spa_common_name',
       'rus_common_name', 'est_common_name', 'urd_common_name', 'calling_code',
       'birth_date'],
      dtype='object')
Index(['Q1', 'A1', 'Q2', 'A2', 'Question', 'Answer', 'birth_country',
       'capital', 'person', 'category', 'person_id', 'rounded_lat',
       'rounded_lng', 'tld', 'ccn3', 'currency', 'currency_short',
       'currency_symbol', 'jpn_common_name', 'spa_common_name',
       'rus_common_name', 'est_common_name', 'urd_common_name', 'calling_code',
       'birth_date'],
      dtype='object')


KeyError: 'callingcode'

In [14]:
import random

def sample_distractor(label_space, ans):
    dis = ans
    while dis == ans:
        dis = random.sample(label_space, 1)[0]
    return dis

def make_choices(ex):
    category_name = ex['category']
    cat1, cat2 = category_name.split('_')

    ans1 = ex['A1']
    ans2 = ex['A2']

    cat1_labelspace = label_space_all[cat1]
    cat2_labelspace = label_space_all[cat2]

    dis1 = sample_distractor(cat1_labelspace, ans1)
    dis2 = sample_distractor(cat2_labelspace, ans2)

    ex['D1'] = dis1
    ex['D2'] = dis2

    #candidates = [ans1, ans2, dis1, dis2]
    #candidates = [dis2, ans2]
    candidates = [ans1, ans2]
    random.shuffle(candidates)

    ans_idx = candidates.index(ans2)

    #print('Q: %s\n(A) %s (B) %s (C) %s (D) %s\nA: %d\n' % (ex['Question'], *candidates, ans_idx))
    print('Q: %s\n(A) %s (B) %s\nA: %d\n' % (ex['Question'], *candidates, ans_idx))

    #for cand_label, cand in zip(['(A)', '(B)', '(C)', '(D)'], candidates):
    for cand_label, cand in zip(['(A)', '(B)'], candidates):
        ex[cand_label] = cand
    
    ex['answer_idx'] = ans_idx

    return ex

df = df.apply(make_choices, axis=1)

Q: What is the capital of the birthplace of Michaela DePrince?
(A) Sierra Leone (B) Freetown
A: 1

Q: What is the capital of the birthplace of Vladimir Lenin?
(A) Moscow (B) Russia
A: 0

Q: What is the capital of the birthplace of Garcelle Beauvais?
(A) Port-au-Prince (B) Haiti
A: 0

Q: What is the capital of the birthplace of Dan Bilzerian?
(A) United States (B) Washington D.C.
A: 1

Q: What is the capital of the birthplace of Shakira?
(A) Bogotá (B) Colombia
A: 0

Q: What is the capital of the birthplace of Benicio Del Toro?
(A) San Juan (B) Puerto Rico
A: 0

Q: What is the capital of the birthplace of Hristo Stoichkov?
(A) Sofia (B) Bulgaria
A: 0

Q: What is the capital of the birthplace of Marie Curie?
(A) Warsaw (B) Poland
A: 0

Q: What is the capital of the birthplace of Ayesha Gaddafi?
(A) Libya (B) Tripoli
A: 1

Q: What is the capital of the birthplace of Alan Walker?
(A) Oslo (B) Norway
A: 0

Q: What is the capital of the birthplace of Jean-Jacques Rousseau?
(A) Bern (B) Switz

In [145]:
#df_qa = df[['Question', 'Answer', '(A)', '(B)', '(C)', '(D)', 'answer_idx']]

#df_qa = df[['Question', 'Answer', 'A1', 'A2']]

df_qa = df[['Question', '(A)', '(B)', 'answer_idx']]

df_qa = df_qa.rename(columns={'Question': 'question', 'answer_idx': 'label', '(A)': 'answer 1', '(B)': 'answer 2'})

df_qa.head()

,question,answer 1,answer 2,label
179,What is the capital of the birthplace of Adoni...,Haiti,Port-au-Prince,1
286,What is the capital of the birthplace of Mala ...,India,New Delhi,1
387,What is the capital of the birthplace of Alek ...,South Sudan,Juba,1
388,What is the capital of the birthplace of Luol ...,Sudan,Khartoum,1
184,What is the capital of the birthplace of Teofi...,Washington D.C.,United States,0


In [160]:
def next_leap_year(year):
    return year + (4 - (year % 4))

next_leap_year(16)

20

In [202]:
arithmetic_ops = [('subtracted by %d' % d, -d) for d in range(1, 11)] + [('added to %d' % d, d) for d in range(1, 11)]

def add_arith_to_question(ex):
    suffix, diff = random.sample(arithmetic_ops, 1)[0]
    
    ex['question'] = ex['question'][:-1] + ', ' + suffix + '?'
    ex['answer 2'] = int(ex['answer 1']) + diff
    ex['label'] = 1

    return ex

In [203]:
year_data = {
   'question': df['Q1'].to_list(),
   'answer 1': df['A1'].to_list()
}
df_year = pd.DataFrame(year_data)

df_year = df_year.apply(add_arith_to_question, axis=1)

df_year.head()

,question,answer 1,answer 2,label
0,"In what year was Zion Williamson born, added t...",2000,2005,1
1,"In what year was Naomi Scott born, added to 7?",1993,2000,1
2,"In what year was Larry Bird born, added to 4?",1956,1960,1
3,"In what year was Donald Glover born, subtracte...",1983,1979,1
4,"In what year was Yara Shahidi born, added to 4?",2000,2004,1


In [209]:
for i in range(100):
    print('%s %s %s' % (df_year['question'][i], df_year['answer 1'][i], df_year['answer 2'][i]))

In what year was Zion Williamson born, added to 5? 2000 2005
In what year was Naomi Scott born, added to 7? 1993 2000
In what year was Larry Bird born, added to 4? 1956 1960
In what year was Donald Glover born, subtracted by 4? 1983 1979
In what year was Yara Shahidi born, added to 4? 2000 2004
In what year was Nicole Brown Simpson born, subtracted by 4? 1959 1955
In what year was Rodrigo Duterte born, subtracted by 7? 1945 1938
In what year was Dylan O’Brien born, added to 9? 1991 2000
In what year was Kenny Rogers born, added to 8? 1938 1946
In what year was Priah Ferguson born, subtracted by 1? 2006 2005
In what year was David Letterman born, subtracted by 1? 1947 1946
In what year was Lulu Wilson born, subtracted by 9? 2005 1996
In what year was Ariel Winter born, added to 5? 1998 2003
In what year was Paul Rudd born, subtracted by 6? 1969 1963
In what year was Kim Kardashian born, subtracted by 4? 1980 1976
In what year was Tyler, The Creator born, added to 1? 1991 1992
In what ye

In [216]:
def shuffle_answers(ex):
    ans1 = ex['answer 1']
    ans2 = ex['answer 2']

    answers = [ans1, ans2]

    random.shuffle(answers)

    new_label = answers.index(ans2)

    ex['answer 1'] = answers[0]
    ex['answer 2'] = answers[1]
    ex['label'] = new_label
    
    return ex

df_year = df_year.apply(shuffle_answers, axis=1)

In [212]:
len(df_qa)

468

In [217]:
df_year.to_csv('datasets/complete_ten/rawdata_ccyear_256.csv', index=False)